# Vector stores and retrievers
- Store data in the form of vectors in vector database and retrieve the same
- Integrate with LLM workflows
- Important for applications that use data to be used by LLM for the purpose of inference. RAG applications benefit from such databases

In [ ]:
import getpass
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

In [2]:
# pip install langchain
# pip install -qU langchain-mistralai


In [3]:
os.environ["MISTRAL_API_KEY"] = getpass.getpass()

from langchain_mistralai import ChatMistralAI

model = ChatMistralAI(model="mistral-large-latest")

 ········


In [4]:
# !pip install langchain-chroma

In [9]:
os.environ['HF_TOKEN'] = getpass.getpass()

 ········


## Documents
- langchain implements a Document class
- Document class holds the text and its metadata
- attributes
  - page_content: text of the document
  - metadata: dict:  data about source of document, its relation with other documents etc..

In [5]:
from langchain_core.documents import Document

documents = [
    Document(
        page_content="Dogs are great companions, known for their loyalty and friendliness.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Goldfish are popular pets for beginners, requiring relatively simple care.",
        metadata={"source": "fish-pets-doc"},
    ),
    Document(
        page_content="Parrots are intelligent birds capable of mimicking human speech.",
        metadata={"source": "bird-pets-doc"},
    ),
    Document(
        page_content="Rabbits are social animals that need plenty of space to hop around.",
        metadata={"source": "mammal-pets-doc"},
    ),
]

## Vector Store
### Vector Search
  - A way of searching text with the help of vector similarity from a store where text is embeded in vectors and stored
### Vector Store
  - A database that stores vectors and provides database operations
  - VectorStore class in langchain provides methods to add, edit, search, upsert Documents in vector store
  - Init with embbeding models
  - Integrations with multiple vector store providers like Postgres or others, hosted on cloud or on premise or in-memory
  - Current example we will use Chroma with its in-memory implementation. Chroma also has capability to persist data.

In [10]:
from langchain_chroma import Chroma
from langchain_mistralai import MistralAIEmbeddings

vectorstore = Chroma.from_documents(
    documents,
    embedding=MistralAIEmbeddings(),
)

Calling .from_documents here will add the documents to the vector store. VectorStore implements methods for adding documents that can also be called after the object is instantiated. Most implementations will allow you to connect to an existing vector store-- e.g., by providing a client, index name, or other information. See the documentation for a specific integration for more detail.

Once we've instantiated a VectorStore that contains documents, we can query it. VectorStore includes methods for querying:

- Synchronously and asynchronously;
- By string query and by vector;
- With and without returning similarity scores;
- By similarity and maximum marginal relevance (to balance similarity with query to diversity in retrieved results).

The methods will generally include a list of Document objects in their outputs.

### Examples
Return documents based on similarity to a string query:

In [11]:
vectorstore.similarity_search("cat")

[Document(page_content='Cats are independent pets that often enjoy their own space.', metadata={'source': 'mammal-pets-doc'}),
 Document(page_content='Cats are independent pets that often enjoy their own space.', metadata={'source': 'mammal-pets-doc'}),
 Document(page_content='Cats are independent pets that often enjoy their own space.', metadata={'source': 'mammal-pets-doc'}),
 Document(page_content='Dogs are great companions, known for their loyalty and friendliness.', metadata={'source': 'mammal-pets-doc'})]

In [12]:
# Async
await vectorstore.asimilarity_search("cat")

[Document(page_content='Cats are independent pets that often enjoy their own space.', metadata={'source': 'mammal-pets-doc'}),
 Document(page_content='Cats are independent pets that often enjoy their own space.', metadata={'source': 'mammal-pets-doc'}),
 Document(page_content='Cats are independent pets that often enjoy their own space.', metadata={'source': 'mammal-pets-doc'}),
 Document(page_content='Dogs are great companions, known for their loyalty and friendliness.', metadata={'source': 'mammal-pets-doc'})]

In [13]:
# Return with scores
vectorstore.similarity_search_with_score("cat")

[(Document(page_content='Cats are independent pets that often enjoy their own space.', metadata={'source': 'mammal-pets-doc'}),
  0.4571284055709839),
 (Document(page_content='Cats are independent pets that often enjoy their own space.', metadata={'source': 'mammal-pets-doc'}),
  0.4571284055709839),
 (Document(page_content='Cats are independent pets that often enjoy their own space.', metadata={'source': 'mammal-pets-doc'}),
  0.4571284055709839),
 (Document(page_content='Dogs are great companions, known for their loyalty and friendliness.', metadata={'source': 'mammal-pets-doc'}),
  0.7032201290130615)]

In [15]:
# Return documents based on similarity to a embedded query
embeddings = MistralAIEmbeddings().embed_query("cat")
vectorstore.similarity_search_by_vector(embeddings)

[Document(page_content='Cats are independent pets that often enjoy their own space.', metadata={'source': 'mammal-pets-doc'}),
 Document(page_content='Cats are independent pets that often enjoy their own space.', metadata={'source': 'mammal-pets-doc'}),
 Document(page_content='Cats are independent pets that often enjoy their own space.', metadata={'source': 'mammal-pets-doc'}),
 Document(page_content='Dogs are great companions, known for their loyalty and friendliness.', metadata={'source': 'mammal-pets-doc'})]

## Retrievers
- VectorStore is not Runnable
- Cannot be used in chain
- LangChain Retrievers are Runnables, so they implement a standard set of methods (e.g., synchronous and asynchronous invoke and batch operations) and are designed to be incorporated in LCEL chains.

We can create a simple version with out subclassing Retriever

In [16]:
from typing import List

from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda

retriever = RunnableLambda(vectorstore.similarity_search).bind(k=1)  # select top result

retriever.batch(["cat", "shark"])

[[Document(page_content='Cats are independent pets that often enjoy their own space.', metadata={'source': 'mammal-pets-doc'})],
 [Document(page_content='Goldfish are popular pets for beginners, requiring relatively simple care.', metadata={'source': 'fish-pets-doc'})]]

Vectorstores implement an as_retriever method that will generate a Retriever, specifically a VectorStoreRetriever. These retrievers include specific search_type and search_kwargs attributes that identify what methods of the underlying vector store to call, and how to parameterize them. For instance, we can replicate the above with the following:

In [18]:
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 1},
)

retriever.batch(["cat", "shark"])

[[Document(page_content='Cats are independent pets that often enjoy their own space.', metadata={'source': 'mammal-pets-doc'})],
 [Document(page_content='Goldfish are popular pets for beginners, requiring relatively simple care.', metadata={'source': 'fish-pets-doc'})]]

VectorStoreRetriever supports search types of "similarity" (default), "mmr" (maximum marginal relevance, described above), and "similarity_score_threshold". We can use the latter to threshold documents output by the retriever by similarity score.

Retrievers can easily be incorporated into more complex applications, such as retrieval-augmented generation (RAG) applications that combine a given question with retrieved context into a prompt for a LLM. Below we show a minimal example.

In [19]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

message = """
Answer this question using the provided context only.

{question}

Context:
{context}
"""

prompt = ChatPromptTemplate.from_messages([("human", message)])

rag_chain = {"context": retriever, "question": RunnablePassthrough()} | prompt | model

In [22]:
response = rag_chain.invoke("tell me about cats")
print(response.content)

Cats are independent pets that often enjoy their own space. This information is sourced from a document on mammal pets.


Retrieval strategies can be rich and complex. For example:

We can infer hard rules and filters from a query (e.g., "using documents published after 2020");
We can return documents that are linked to the retrieved context in some way (e.g., via some document taxonomy);
We can generate multiple embeddings for each unit of context;
We can ensemble results from multiple retrievers;
We can assign weights to documents, e.g., to weigh recent documents higher.
The retrievers section of the how-to guides covers these and other built-in retrieval strategies.

It is also straightforward to extend the BaseRetriever class in order to implement custom retrievers. See our how-to guide here.

# Process Flow
```mermaid
flowchart LR
A[Document from text] --> B(List of Documents)
B-->C(VectorStore from Documents)
C-->D(Retrievers)
D-->E(Chain)
```